In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from sklearn.linear_model import LogisticRegression
import scipy.stats as st
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split,KFold,StratifiedKFold,GridSearchCV,RandomizedSearchCV,cross_val_score
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import r2_score,roc_auc_score,recall_score,classification_report,mean_squared_error,accuracy_score,confusion_matrix

In [6]:
#loading the data
data_train_smote=pd.read_csv("full_train_after_smote.csv")
data_test_smote=pd.read_csv("cleaned_data_test_feature_engineered.csv")


#dropping the feature engineered data from test
data_test_smote=data_test_smote.drop(["YEAR","INCIDENT_ID","MONTH","DAY","DATE"],axis=1)
data_train_smote=data_train_smote.drop(["INCIDENT_ID"],axis=1)

print(data_train_smote.shape,data_test_smote.shape)


#splitting into train and validation
train,val=train_test_split(data_train_smote,test_size=.20,random_state=42)
train.shape,val.shape


#splitting into x and y
Xfull=data_train_smote.drop(["MULTIPLE_OFFENSE"],axis=1)
yfull=data_train_smote["MULTIPLE_OFFENSE"]
print(Xfull.shape,yfull.shape)


Xtrain=train.drop(["MULTIPLE_OFFENSE"],axis=1)
ytrain=train["MULTIPLE_OFFENSE"]
print(Xtrain.shape,ytrain.shape)

Xval=val.drop(["MULTIPLE_OFFENSE"],axis=1)
yval=val["MULTIPLE_OFFENSE"]
print(Xval.shape,yval.shape)


((45572, 16), (15903, 15))

In [13]:
lrc = LogisticRegression(class_weight="balanced")
rfc=RandomForestClassifier(class_weight="balanced")
adbc=AdaBoostClassifier()
bgc=BaggingClassifier()
gnb = GaussianNB()
knn=KNeighborsClassifier()
dtc = DecisionTreeClassifier()
bgcl_lrc = BaggingClassifier(base_estimator=lrc, random_state=42)
ab_rfc = AdaBoostClassifier(base_estimator=rfc,random_state=42)
ab_dtc = AdaBoostClassifier(base_estimator=dtc,random_state=42)
ab_nbc=  AdaBoostClassifier(base_estimator=gnb,random_state=42)
ab_lrc=  AdaBoostClassifier(base_estimator=lrc,random_state=42)
gbc=GradientBoostingClassifier()
ab_gbc=  AdaBoostClassifier(base_estimator=gbc,random_state=42)
xgbc=XGBClassifier()
ab_xgbc=  AdaBoostClassifier(base_estimator=xgbc,random_state=42)

In [ ]:


models=[lrc,rfc,adbc,bgc,gnb,knn,dtc,bgcl_lrc,ab_rfc,ab_dtc,ab_nbc,ab_lrc,gbc,ab_gbc,xgbc,ab_xgbc]
sctr,scte,auc,ps,rs=[],[],[],[],[]
def ensemble_for_train_val(X_train,X_test, y_train, y_test):
    for model in models:
            model.fit(X_train, y_train)
            y_test_pred = model.predict(X_test)
            y_test_pred_new=model.predict_proba(X_test)
            y_test_pred_new=y_test_pred_new[:,1]
            train_score=model.score(X_train,y_train)
            test_score=model.score(X_test,y_test)
            p_score=metrics.precision_score(y_test,y_test_pred)
            r_score=metrics.recall_score(y_test,y_test_pred)
            
            ac=metrics.roc_auc_score(y_test,y_test_pred_new)
            
            sctr.append(train_score)
            scte.append(test_score)
            ps.append(p_score)
            rs.append(r_score)
            auc.append(ac)
    return sctr,scte,auc,ps,rs


#checking the performance on train and validation
ensemble_for_train_val(Xtrain,Xval, ytrain, yval)
# 'ab_rf','ab_dt','ab_nb','ab_lr','bgcl_lr'
ensemble=pd.DataFrame({'names':['Logistic Regression','Random Forest','Ada boost','Bagging',
                                'Naive-Bayes','KNN','Decistion Tree',
                                'bagged LR',"adaboost rf","adaboost dtc","adaboost naive bayes",
                                "adaboost logistic regression","gradient boosting trees","adaboost gbc","xgboost","adaboost xgbc"],
                       'auc_score':auc,'training':sctr,'testing':scte,'precision':ps,'recall':rs})
ensemble=ensemble.sort_values(by='auc_score',ascending=False).reset_index(drop=True)
ensemble

In [ ]:
ensemble=ensemble.sort_values(by='auc_score',ascending=False).reset_index(drop=True)
ensemble